In [9]:
import collections
import re
from d2l import torch as d2l


In [3]:
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt',
                                '090b5e7e70c295757f55df93cb0a180b9691891a')

In [6]:
#将数据集读取到由多条文本组成的列表中
def read_time_machine():
    with open(d2l.download('time_machine'),'r') as f:
        lines = f.readlines()
    return [re.sub('[^A-Za-z]+',' ',line).strip().lower() for line in lines]


lines = read_time_machine()
print(f'# 文本总行数: {len(lines)}')
print(lines[0])
print(lines[10])


# 文本总行数: 3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


In [8]:
#每个文本序列又被拆成一个标记列表

def tokenize(lines,token='word'):
    #将文本拆分成单词或者字符标记
    if token == 'word':    #toekn ----  词元
        return [line.split() for line in lines]   #split() 通过指定分隔符对字符串进行切片，
        #如果参数 num 有指定值，则分隔 num+1 个子字符串语法
    elif token == 'char':
        return [list(line) for line in lines]
    else:
        print('错误' + token)

tokens = tokenize(lines)   #tokens，，分好组的词元列表，，，，是一个列表
for i in range(11):
    print(tokens[i])


['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


In [11]:
def count_corpus(tokens):  #@save
    """统计词元的频率"""
    # 这里的tokens是1D列表或2D列表
    if len(tokens) == 0 or isinstance(tokens[0], list):  #isinstance() 函数来判断一个对象是否是一个已知的类型，类似 type()
        # 将词元列表展平成一个列表
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)   #counter-----统计每个元素出现的次数


In [17]:
#构建一个字典或者叫词汇表（vocabulary），用来将字符串类型的标记映射到从0开始的数字索引中

class Vocab:
    def __init__(self,tokens=None,min_freq=0,reserved_token=None):
        #min_freq,如果一个词少于这个次数，就丢掉。reserved_token标记开始和结束
        if tokens is None:
            tokens = []
        if reserved_token is None:
            reserved_token = []
            
        # 按出现频率排序
        counter = count_corpus(tokens)
        self.token_freqs = sorted(counter.items(),key=lambda x: x[1],#items() 函数以列表返回可遍历的(键, 值) 元组数组
                                   reverse=True)
                                   #sorted(iterable, cmp=None, key=None, reverse=False)
                                   #iterable -- 可迭代对象。cmp -- 比较的函数，key -- 主要是用来进行比较的元素
                                   # reverse -- 排序规则，reverse = True 降序 ， reverse = False 升序（默认），
        self.unk,uniq_tokens = 0,['<unk>'] + reserved_token   #unk 代表一个未知的token，下标为0；
                                                              #将uniq_token，独一无二的token存入unk
        uniq_tokens += [
                        token for token,freq in self.token_freqs
                        if freq >= min_freq and token not in uniq_tokens   
                        #如果出现的频率大于min_freq且不在字典里，就把它放入字典
        ]
        self.idx_to_token,self.token_to_idx = [],dict()   #给idx返回字典，给字典返回idx 【】列表---  dict() 字典键值对
        for token in uniq_tokens:
            self.idx_to_token.append(token)  # 把uniq_token里的单词放到列表里   append() 方法用于在列表末尾添加新的对象
            self.token_to_idx[token] = len(self.idx_to_token) - 1  #单词和索引一一对应


    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self,tokens):   #作用是给一个token返回index
        if not isinstance(tokens,(list,tuple)):
            return self.token_to_idx.get(tokens,self.unk)
        return [self.__getitem__(token) for token in tokens]



    def to_tokens(self,indices):
        if not isinstance(indices,(list,tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

In [18]:
#构建词汇表

vocab = Vocab(tokens)
print(list(vocab.token_to_idx.items())[:10])

[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]


In [19]:
#将每一条文本转化成一个数字索引

for i in [0,10]:
    print('words',tokens[i])
    print('indices:',vocab[tokens[i]])

words ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
indices: [1, 19, 50, 40, 2183, 2184, 400]
words ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
indices: [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


In [22]:
#功能整合到一起

def load_corpus_time_machine(max_tokens=-1):  #@save
    """返回时光机器数据集的词元索引列表和词表"""
    lines = read_time_machine()
    tokens = tokenize(lines, 'char')
    vocab = Vocab(tokens)
    # 因为时光机器数据集中的每个文本行不一定是一个句子或一个段落，
    # 所以将所有文本行展平到一个列表中
    corpus = [vocab[token] for line in tokens for token in line]
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus, vocab

corpus, vocab = load_corpus_time_machine()
len(corpus), len(vocab)   #corpus语料

(170580, 28)

: 